In [ ]:
from Korpora import Korpora    # 데이터

import pandas as pd
import re                      # 정규표현식

from konlpy.tag import Okt

# from nltk.corpus import stopwords  # 불용어 지우기 위한 임포트
# import nltk

from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
nsmc = Korpora.load('nsmc')


    Korpora 는 다른 분들이 연구 목적으로 공유해주신 말뭉치들을
    손쉽게 다운로드, 사용할 수 있는 기능만을 제공합니다.

    말뭉치들을 공유해 주신 분들에게 감사드리며, 각 말뭉치 별 설명과 라이센스를 공유 드립니다.
    해당 말뭉치에 대해 자세히 알고 싶으신 분은 아래의 description 을 참고,
    해당 말뭉치를 연구/상용의 목적으로 이용하실 때에는 아래의 라이센스를 참고해 주시기 바랍니다.

    # Description
    Author : e9t@github
    Repository : https://github.com/e9t/nsmc
    References : www.lucypark.kr/docs/2015-pyconkr/#39

    Naver sentiment movie corpus v1.0
    This is a movie review dataset in the Korean language.
    Reviews were scraped from Naver Movies.

    The dataset construction is based on the method noted in
    [Large movie review dataset][^1] from Maas et al., 2011.

    [^1]: http://ai.stanford.edu/~amaas/data/sentiment/

    # License
    CC0 1.0 Universal (CC0 1.0) Public Domain Dedication
    Details in https://creativecommons.org/publicdomain/zero/1.0/

[Korpora] Corpus `nsmc` is already installed at C:\Users\a\Korpora\nsmc\ratings_train.txt
[Korpora] Corpus `nsmc` is already installed at C:\Users\a\Korpo

In [ ]:
nsmcDF = pd.DataFrame(nsmc.test)
nsmcDF.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    50000 non-null  object
 1   label   50000 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 781.4+ KB


##### 한국어만 남겨

- 영어는 필요없다 싹 다 지운다. 왜냐하면 내가 그러고 싶기 때문이다.

In [ ]:
def test(s):
    hangul = re.compile('[^ ㄱ-ㅣ가-힣]+')
    result = hangul.sub('', s)
    return result

nsmcDF['Q_refined'] = nsmcDF.apply(lambda row: test(row['text']), axis=1)

In [ ]:
nsmcDF.head(5)

,text,label,Q_refined
0,굳 ㅋ,1,굳 ㅋ
1,GDNTOPCLASSINTHECLUB,0,
2,뭐야 이 평점들은.... 나쁘진 않지만 10점 짜리는 더더욱 아니잖아,0,뭐야 이 평점들은 나쁘진 않지만 점 짜리는 더더욱 아니잖아
3,지루하지는 않은데 완전 막장임... 돈주고 보기에는....,0,지루하지는 않은데 완전 막장임 돈주고 보기에는
4,3D만 아니었어도 별 다섯 개 줬을텐데.. 왜 3D로 나와서 제 심기를 불편하게 하죠??,0,만 아니었어도 별 다섯 개 줬을텐데 왜 로 나와서 제 심기를 불편하게 하죠


##### 명사 추출
- 자꾸 터져서 그냥 다르게 진행
- 명사만 남긴 새로운 컬럼을 만들어본다

In [ ]:
from konlpy.tag import Okt

okt = Okt()

nsmcDF['token'] = nsmcDF.apply(lambda row: okt.nouns(row['Q_refined']), axis=1)


- 보면 알겠지만 굳은 인식 안되서 삭제 .. 시간이 없으므로 일단 그냥 진행

In [ ]:
nsmcDF.head(5)

,text,label,Q_refined,token
0,굳 ㅋ,1,굳 ㅋ,[]
1,GDNTOPCLASSINTHECLUB,0,,[]
2,뭐야 이 평점들은.... 나쁘진 않지만 10점 짜리는 더더욱 아니잖아,0,뭐야 이 평점들은 나쁘진 않지만 점 짜리는 더더욱 아니잖아,"[뭐, 이, 평점, 점, 리, 더, 더욱]"
3,지루하지는 않은데 완전 막장임... 돈주고 보기에는....,0,지루하지는 않은데 완전 막장임 돈주고 보기에는,"[완전, 막장, 임, 돈, 보기]"
4,3D만 아니었어도 별 다섯 개 줬을텐데.. 왜 3D로 나와서 제 심기를 불편하게 하죠??,0,만 아니었어도 별 다섯 개 줬을텐데 왜 로 나와서 제 심기를 불편하게 하죠,"[만, 별, 다섯, 개, 왜, 로, 제, 심기]"


##### NLTK 불용어말고 txt 사용
- NLTK 불용어를 사용하려다가 외부에 잇는걸 다운받아서 사용하기로 결정했다

In [ ]:
stopwords_path = r'C:\Users\a\Desktop\자연어처리\24_10_07\stopwords-ko.txt'

In [ ]:
with open(stopwords_path, 'r', encoding='utf-8') as file:
    stopwords = file.read().splitlines()

In [ ]:
# 不用語 제거 함수
def remove_stopwords(tokens):
    return [token for token in tokens if token not in stopwords]

In [ ]:
nsmcDF['token'] = nsmcDF['token'].apply(lambda tokens: remove_stopwords(tokens))

In [ ]:
nsmcDF

,text,label,Q_refined,token
0,굳 ㅋ,1,굳 ㅋ,[]
1,GDNTOPCLASSINTHECLUB,0,,[]
2,뭐야 이 평점들은.... 나쁘진 않지만 10점 짜리는 더더욱 아니잖아,0,뭐야 이 평점들은 나쁘진 않지만 점 짜리는 더더욱 아니잖아,"[뭐, 평점, 점, 리, 더, 더욱]"
3,지루하지는 않은데 완전 막장임... 돈주고 보기에는....,0,지루하지는 않은데 완전 막장임 돈주고 보기에는,"[완전, 막장, 임, 돈, 보기]"
4,3D만 아니었어도 별 다섯 개 줬을텐데.. 왜 3D로 나와서 제 심기를 불편하게 하죠??,0,만 아니었어도 별 다섯 개 줬을텐데 왜 로 나와서 제 심기를 불편하게 하죠,"[만, 별, 개, 심기]"
...,...,...,...,...
49995,오랜만에 평점 로긴했네ㅋㅋ 킹왕짱 쌈뽕한 영화를 만났습니다 강렬하게 육쾌함,1,오랜만에 평점 로긴했네ㅋㅋ 킹왕짱 쌈뽕한 영화를 만났습니다 강렬하게 육쾌함,"[만, 평점, 킹왕짱, 쌈뽕, 영화, 쾌함]"
49996,의지 박약들이나 하는거다 탈영은 일단 주인공 김대희 닮았고 이등병 찐따 OOOO,0,의지 박약들이나 하는거다 탈영은 일단 주인공 김대희 닮았고 이등병 찐따,"[의지, 박약, 탈영, 주인공, 김대희, 이등병]"
49997,그림도 좋고 완성도도 높았지만... 보는 내내 불안하게 만든다,0,그림도 좋고 완성도도 높았지만 보는 내내 불안하게 만든다,"[그림, 완성, 도도, 내내]"
49998,절대 봐서는 안 될 영화.. 재미도 없고 기분만 잡치고.. 한 세트장에서 다 해먹네,0,절대 봐서는 안 될 영화 재미도 없고 기분만 잡치고 한 세트장에서 다 해먹네,"[절대, 안, 영화, 재미, 기분, 잡, 세트, 해먹]"


##### 토큰을 숫자로
- 나에겐 시간이 없으므로 되든 안되든 진행

In [ ]:
# 토큰 공백으로 붙여서 형태 변환 진행
nsmcDF['token_joined'] = nsmcDF['token'].apply(lambda tokens: ' '.join(tokens))

In [ ]:
# 벡터화 벡터화
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(nsmcDF['token_joined'])

In [ ]:
X_df = pd.DataFrame(X.toarray(), columns=vectorizer.get_feature_names_out())

In [ ]:
X_df

,가가,가각,가감,가게,가격,가계,가공,가관,가교,가구,...,힘들떄,힘일,힘줄,힙겨웠다,힙스터,힙합,힛걸,힛쳐,힛츄,힛힛
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
49996,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
49997,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
49998,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


##### 끝
- 딱봐도 이상하지만 시간이 없으므로 여기까지 ...고치는건 여러분들에게 맡기겠습니다.

-   패딩 안맞춤. ㅈㅅ